# import

In [ ]:
from os import rename
from pathlib import Path

import pandas as pd

# Dataframes

## from dict

In [ ]:
data = {
    'Country': ['Belgium', 'India', 'Brazil'],
    'Capital': ['Brussels', 'New Delhi', 'Brasilia'],
    'Population': [100, 200, 300]
}
df = pd.DataFrame(data, columns=['Country', 'Capital', 'Population'])

## from files

### CSV

In [ ]:
# Simple read file
df = pd.read_csv("../data/products.csv")

"""
Most commons params

| Parameter                   | Description                                                | Common Use Case                                             |
| --------------------------- | ---------------------------------------------------------- | ----------------------------------------------------------- |
| filepath_or_buffer          | File path (string or URL) or file-like object              | Required to point to the CSV file                           |
| sep                         | Field delimiter (default is `','`)                         | Use `\t` for TSV, `;` for European-style CSV                |
| header                      | Row number to use as column names (default is `0`)         | Use `None` if there's no header                             |
| names                       | List of column names to use (overrides `header`)           | Useful when header is missing or needs renaming             |
| index_col                   | Column(s) to set as index                                  | Often used to set time or ID columns as index               |
| usecols                     | Subset of columns to read (by name or index)               | Improves performance when not all columns are needed        |
| dtype                       | Dict of column types                                       | Enforces type consistency or memory optimization            |
| parse_dates                 | List of columns to parse as datetime                       | Automatically converts date strings into `datetime` objects |
| infer_datetime_format       | Speed up datetime parsing when format is consistent        | Faster parsing with consistent formats                      |
| na_values                   | List of strings to recognize as NaN                        | Customize missing value handling (`["NA", "NULL", "n/a"]`)  |
| skiprows                    | Number of lines to skip at the start                       | Useful for skipping metadata or junk rows                   |
| nrows                       | Number of rows to read                                     | Used for sampling large files                               |
| chunksize                   | Return an iterator with chunks of data (number of lines)   | Enables streaming large datasets in pieces                  |
| encoding                    | Encoding of the file (default is `'utf-8'`)                | Use `'ISO-8859-1'` or `'latin1'` for special characters     |
| compression                 | For compressed files (`'gzip'`, `'zip'`, etc.)             | Automatically handles zipped data files                     |

"""



In [ ]:
"""
    This would read a CSV in chunks from S3,
    parse timestamps,
    optimize memory by enforcing types,
    and limit to specific useful columns.
"""
df = pd.read_csv(
    "s3://bucket/data.csv",
    sep=",",
    usecols=["id", "timestamp", "event_type"],
    parse_dates=["timestamp"],
    dtype={"id": "int32", "event_type": "category"},
    chunksize=100000
)

In [ ]:
"""
    Treating date fields
"""
df = pd.read_csv(f"{Path().absolute()}/netflix_titles.csv",
                 parse_dates=['date_added'],
                 date_format='%B %d, %Y')

# It's required to use str.strip() because some of the values
# have a whitespace at the beginning
df['date_added'] = pd.to_datetime(df['date_added'].str.strip(), format='%B %d, %Y')

df.dtypes

## from database

In [ ]:
from sqlalchemy import create_engine

# Database connection settings
host = "your_host"
port = "5432"
database = "your_database"
user = "your_username"
password = "your_password"

# Create the SQLAlchemy connection string
connection_string = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"

# Create the database engine
engine = create_engine(connection_string)

# Example query
query = """
        SELECT id, name, created_at
        FROM your_table
        WHERE created_at >= CURRENT_DATE - INTERVAL '30 days' \
        """

# Read the query into a DataFrame
df = pd.read_sql(query, engine)

# Preview data
print(df.head())


## basic info

In [ ]:
# df.shape   # (rows,columns)
# df.index   # Describe index
# df.columns # Describe columns
# df.info()  # Info of df
df.count()  # Number of non-NA values

# Saving dataframes

## to files

In [ ]:
df.to_csv('data.csv', index=False)

"""
| Parameter             | Description                                           | Common Use Case                                         |
| --------------------- | ----------------------------------------------------- | ------------------------------------------------------- |
| path_or_buf           | File path or file-like object                         | Local path or cloud storage path                        |
| sep                   | Field delimiter (default is `','`)                    | Use `\t` for TSV or `;` in European formats             |
| index                 | Whether to write row indices (default `True`)         | Set to `False` to exclude index column                  |
| columns               | List of columns to write                              | Export only selected columns                            |
| header                | Whether to write column names (default `True`)        | Set to `False` to exclude header row                    |
| mode                  | Write mode (`'w'` or `'a'`)                           | `'a'` allows appending to existing files                |
| encoding              | Output encoding (default `'utf-8'`)                   | Use `'utf-8-sig'` for Excel compatibility or `'latin1'` |
| compression           | Compression format (`'gzip'`, `'zip'`, `'bz2'`, etc.) | Compress files during export                            |
| line_terminator       | End-of-line character (`'\n'`, `'\r\n'`)              | Control line endings across platforms                   |
| float_format          | Format for floating-point numbers                     | Useful for consistent rounding (e.g., `"%.2f"`)         |
| date_format           | Format for datetime columns                           | Standardize timestamp export (`"%Y-%m-%d"`)             |
| quoting               | CSV quoting behavior (from `csv` module)              | Handle special characters in text                       |
| chunksize             | Number of rows to write at a time                     | Useful for large data export                            |
| na_rep                | Representation for NaN/missing values                 | Replace NaN with specific text (e.g., `'-'`)            |

"""

# Tranformations

## drop columns

In [ ]:
df2 = df.drop(columns=['Capital'])
df2

## drop rows

In [ ]:
df2 = df.drop(0)  # index-based, drops first row
df2

## applying functions

In [ ]:
f = lambda x: x * 3
df2 = df.apply(f)

In [ ]:
def concat_country_capital(row):
    return f"{row['Country']} ({row['Capital']})"


df2 = df.copy()

# Using axis = 1, so the parameters passed to the function
# is a PD Series representing the row
df2['new_column'] = df2.apply(concat_country_capital, axis=1)
df2

# Combining Data

## df merge

In [ ]:

# reading two csv files
products_df = pd.read_csv("../data/products.csv")
sales_df = pd.read_csv("../data/sales.csv", parse_dates=["sale_date"])

merged_sales_df = pd.merge(products_df, sales_df,
                           on='product_id',
                           how='left')

merged_sales_df

## finding absent values

In [ ]:
# Load the CSV files
products_df = pd.read_csv("../data/products.csv")
sales_df = pd.read_csv("../data/sales.csv")

# Find products without sales
products_with_sales = sales_df["product_id"].unique()

# What doest the trick if the ~ operator, it works like "not in"
products_without_sales = products_df[~products_df["product_id"].isin(products_with_sales)]

print("Products without sales:")
print(products_without_sales)

# Calculations

## Sort by index

In [ ]:
df2 = df.sort_index(ascending=True)

## Sort by values

In [ ]:
df2 = df.sort_values(by='Capital', ascending=True)

## cumulative sum

In [ ]:
df2 = df.copy()

df2['Cumulative_pop'] = df2['Population'].cumsum()

## min, max, mean, median, describe stats

In [ ]:
# df['Population'].min()
# df['Population'].max()
# df['Population'].mean() # = average
# df['Population'].median() # middle value in a sorted dataset
# df['Population'].describe()

## group by and sum

In [ ]:
sales_summary = (merged_sales_df.groupby('product_id')['quantity'].sum()
                 .sort_values(ascending=False)
                 .reset_index())
sales_summary

## finding the highest sale (group, agg)

In [ ]:
# Group sales by product_id and sum the quantity
# Calculate total revenue per product
sales_summary = (
    merged_sales_df
    .groupby(["product_id", "product_name", "category", "price"])
    .agg(total_quantity=("quantity", "sum"),
         total_revenue=("quantity", lambda x: (x * merged_sales_df.loc[x.index, "price"]).sum()))
    .reset_index()
)

# Sort by quantity to find the most sold product
most_sold_product = sales_summary.sort_values(by="total_revenue", ascending=False).head(1)

print("Most sold product with total revenue:")
print(most_sold_product)